In [ ]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner


logger = logging.getLogger(__name__)
global_config = None

logging = logging.getLogger(__name__)
global_config = None

dataset_name = 'lamini_docs.jsonl'
dataset_path = f"/content/{dataset_name}"
use_hf = False

dataset_path = 'lamini/lamini_docs'
use_hf = True  # 是否使用huggingface
model_name = 'EleutherAI/pythia-70m'
training_config = {
    'model': {
        'pretrained_name': model_name,
        'max_length': 2048
    },
    'datasets': {
        'use_hf': use_hf,
        'path': dataset_path
    },
    'verbose': True
}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

In [ ]:
base_model=AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
device_count=torch.cuda.device_count()
if device_count>0:
  logger.debug('Select GPU device')
  device=torch.device('cuda')
else:
  logger.debug('Select CPU device')
  device=torch.device('cpu')


In [ ]:
base_model.to(device)

In [ ]:
def inference(text,model,tokenizer,max_input_tokens=1000,max_output_tokens=100):
  #Tokenize
  input_ids=tokenizer.encode(
      text,
      return_tensors='pt',
      truncation=True,
      max_length=max_input_tokens
  )
  #Generate
  device=model.device
  generated_tokens_with_prompt=model.generate(
      input_ids=input_ids.to(device),
      max_length=max_output_tokens
  )
  #Decode
  generated_text_with_prompt=tokenizer.batch_decode(generated_tokens_with_prompt,skip_special_tokens=True)
  #Strip the prompt
  generated_text_answer=generated_text_with_prompt[0][len(text):]
  return generated_text_answer


In [ ]:
test_text=test_data[0]['question']
print('Question input (test): ',test_text)
print(f'Correct answer from Lamini docs:{test_dataset[0]['answer']}')
print("model's answer:")
print(inference(text_text,base_model,tokenizer))

In [ ]:
max_steps=3
trained_model_name=f"lamini_docs_{max_steps}_steps"
output_dir=trained_model_name

In [ ]:
training_args=TrainingArguments(
      # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,
  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,
        # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)